In [10]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

# Sets
EMPLOYEES = [f'E_{i}' for i in range(1, 16)]
ROLES = ['Barista', 'Cashier', 'Shift_Lead', 'Food_Prep', 'Cleaner']
DAYS = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
SHIFT_TYPES = ['S1', 'S2', 'S3', 'S4']
SHIFTS = [f'{d}_{st}' for d in DAYS for st in SHIFT_TYPES]

# Parameters

# Shift Duration (Hs) - S1, S2, S3 are 4 hours, S4 is 5 hours (7 PM - 12 AM)
SHIFT_HOURS = {
    'S1': 4, 'S2': 4, 'S3': 4, 'S4': 5
}
HOURS_s = {s: SHIFT_HOURS[s.split('_')[1]] for s in SHIFTS}

# Employee Parameters
WAGE_e = {e: 15 for e in EMPLOYEES}  # $15/hour
MAX_HOURS_e = {e: 20 for e in EMPLOYEES} # 20 hours max per week

# Skills Matrix (Ke,r) - 15 Employees x 5 Roles (from simulated data)
# Order: Barista, Cashier, Shift_Lead, Food_Prep, Cleaner
SKILLS_K_er_data = {
    'E_1': [1, 1, 1, 0, 1], 'E_2': [1, 1, 0, 1, 1], 'E_3': [1, 0, 1, 1, 0],
    'E_4': [1, 1, 0, 0, 1], 'E_5': [0, 1, 1, 1, 0], 'E_6': [1, 0, 0, 1, 1],
    'E_7': [1, 1, 0, 0, 1], 'E_8': [0, 1, 1, 1, 0], 'E_9': [1, 0, 1, 0, 1],
    'E_10': [1, 1, 0, 1, 0], 'E_11': [0, 1, 1, 0, 1], 'E_12': [1, 1, 0, 1, 1],
    'E_13': [1, 0, 1, 1, 0], 'E_14': [1, 1, 0, 0, 1], 'E_15': [0, 1, 0, 1, 1]
}
K_er = {
    (e, r): SKILLS_K_er_data[e][i]
    for i, r in enumerate(ROLES) for e in EMPLOYEES
}

# Shift Requirements (Rs,r) - Required employees per role for each shift type
# Order: Barista, Cashier, Shift_Lead, Food_Prep, Cleaner
SHIFT_REQUIREMENTS = {
    'S1': [2, 1, 1, 0, 0],  # 7-11 AM
    'S2': [2, 1, 1, 1, 0],  # 11 AM-3 PM
    'S3': [1, 1, 1, 0, 1],  # 3 PM-7 PM
    'S4': [1, 1, 0, 0, 1]   # 7 PM-12 AM
}
R_sr = {}
for s in SHIFTS:
    day, shift_type = s.split('_')
    for i, r in enumerate(ROLES):
        R_sr[(s, r)] = SHIFT_REQUIREMENTS[shift_type][i]

# Simulated Availability (Ae,s) & Preference (Pe,s)
np.random.seed(3230) # for reproducibility

# Create a base binary availability matrix (15x28)
AVAILABILITY_MATRIX = np.random.randint(0, 2, size=(len(EMPLOYEES), len(SHIFTS)))

# Create preference scores (1-5) when available (0 if not available)
PREFERENCE_MATRIX = np.zeros_like(AVAILABILITY_MATRIX)
for i in range(len(EMPLOYEES)):
    for j in range(len(SHIFTS)):
        if AVAILABILITY_MATRIX[i, j] == 1:
            PREFERENCE_MATRIX[i, j] = np.random.randint(1, 6) # Score 1 to 5

A_es = {} # Availability parameter
P_es = {} # Preference parameter
for i, e in enumerate(EMPLOYEES):
    for j, s in enumerate(SHIFTS):
        A_es[(e, s)] = AVAILABILITY_MATRIX[i, j]
        P_es[(e, s)] = PREFERENCE_MATRIX[i, j]

# Overlapping Shifts
CONFLICTS = []
for day in DAYS:
    # S1 (7-11) conflicts with S2 (11-3)
    CONFLICTS.append((f'{day}_S1', f'{day}_S2'))
    # S2 (11-3) conflicts with S3 (3-7)
    CONFLICTS.append((f'{day}_S2', f'{day}_S3'))
    # S3 (3-7) conflicts with S4 (7-12)
    CONFLICTS.append((f'{day}_S3', f'{day}_S4'))

# Multi-Objective Weight (lambda)
LAMBDA = 0.6  # 60% weight on Cost, 40% on Preference


# MODEL FORMULATION

# Create Gurobi Model
model = gp.Model("CoffeeShop_Scheduling_MILP")

# Decision Variables (xe,s)
# Binary variable: 1 if employee 'e' is assigned to shift 's', 0 otherwise
x = model.addVars(EMPLOYEES, SHIFTS, vtype=GRB.BINARY, name="Assign")

# Objective Function
# Minimize Z = λ * (Total Cost) - (1-λ) * (Total Preference)

# Total Cost component
TotalCost = gp.quicksum(
    x[e, s] * WAGE_e[e] * HOURS_s[s]
    for e in EMPLOYEES for s in SHIFTS
)

# Total Preference component
TotalPreference = gp.quicksum(
    x[e, s] * P_es[e, s]
    for e in EMPLOYEES for s in SHIFTS
)

model.setObjective(
    LAMBDA * TotalCost - (1 - LAMBDA) * TotalPreference,
    GRB.MINIMIZE
)

# Constraints

# Shift Coverage and Skill Requirements
# Each shift 's' must have R_sr required employees for role 'r'
for s in SHIFTS:
    for r in ROLES:
        # Sum of assignments (x[e,s]) for employees 'e' who have skill 'r' (K_er)
        model.addConstr(
            gp.quicksum(x[e, s] * K_er[e, r] for e in EMPLOYEES) >= R_sr[s, r],
            name=f"Req_{s}_{r}"
        )

# Employee Availability
# Employee 'e' can only be assigned to shift 's' if available (A_es = 1)
for e in EMPLOYEES:
    for s in SHIFTS:
        model.addConstr(
            x[e, s] <= A_es[e, s],
            name=f"Avail_{e}_{s}"
        )

# Workload (Max Weekly Hours)
# Employee 'e' total hours must not exceed max hours (M_e=20)
for e in EMPLOYEES:
    model.addConstr(
        gp.quicksum(x[e, s] * HOURS_s[s] for s in SHIFTS) <= MAX_HOURS_e[e],
        name=f"MaxHours_{e}"
    )

# No Overlapping Shifts
# An employee cannot be assigned to two shifts that overlap
for e in EMPLOYEES:
    for s1, s2 in CONFLICTS:
        model.addConstr(
            x[e, s1] + x[e, s2] <= 1,
            name=f"Conflict_{e}_{s1}_{s2}"
        )


In [11]:
print("SIMULATED INPUT DATA TABLES ")

# DISPLAY EMPLOYEE AND ROLE DATA
print("\n Employee Skills and Wages")

# Prepare Employee Skills (K_er) for display
skills_data_rows = []
for e in EMPLOYEES:
    row = {'Employee': e, 'Wage ($)': WAGE_e[e], 'Max Hrs': MAX_HOURS_e[e]}
    for r in ROLES:
        row[r] = K_er.get((e, r), 0)
    skills_data_rows.append(row)

Employee_DF = pd.DataFrame(skills_data_rows)
print("### Employee Skills (1 = Has Skill) and Max Hours")
print(Employee_DF.to_string(index=False)) 

print("\n")
# SHIFT REQUIREMENTS
print("Shift Requirements Data (R_sr)")

# Prepare Shift Requirements
req_data_rows = []
for st in SHIFT_TYPES:
    s = DAYS[0] + '_' + st 
    row = {'Shift Type': st, 'Hours': HOURS_s[s]}
    for r in ROLES:
        row[r] = R_sr.get((s, r), 0)
    req_data_rows.append(row)

Requirements_DF = pd.DataFrame(req_data_rows)
print(" Required Employees per Role and Shift Duration (Applies to all days)")
print(Requirements_DF.to_string(index=False))

# DISPLAY COMPLETE AVAILABILITY AND PREFERENCE MATRICES
print("Complete Availability and Preference Matrices")

# reshape availability and preference dictionaries into dataframes
availability_df_data = []
preference_df_data = []

for e in EMPLOYEES:
    avail_row = {'Employee': e}
    pref_row = {'Employee': e}
    for s in SHIFTS:
        avail_row[s] = A_es[(e, s)]
        pref_row[s] = P_es[(e, s)]
    availability_df_data.append(avail_row)
    preference_df_data.append(pref_row)

Availability_DF = pd.DataFrame(availability_df_data).set_index('Employee')
Preference_DF = pd.DataFrame(preference_df_data).set_index('Employee')

print(" Employee Availability (1 = Available, 0 = Not available)")
# The .to_string() method is used to ensure all rows/columns are printed without truncation
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(Availability_DF.to_string())

print("\n Employee Preference (Score 1-5, 0 = Not available)")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(Preference_DF.to_string())

print("\n")
# DISPLAY SHIFT CONFLICTS
print("Shift Conflicts")
print("An employee cannot work both shifts in a pair on the same day.")
print(f"Total conflict pairs identified: {len(CONFLICTS)}")
print("Full List of Conflict Pairs:")

# dataframe for all conflicts
conflict_rows = [{'Conflict 1': c[0], 'Conflict 2': c[1]} for c in CONFLICTS]
Conflict_DF = pd.DataFrame(conflict_rows)
print(Conflict_DF.to_string(index=False))


print("\n")

SIMULATED INPUT DATA TABLES 

 Employee Skills and Wages
### Employee Skills (1 = Has Skill) and Max Hours
Employee  Wage ($)  Max Hrs  Barista  Cashier  Shift_Lead  Food_Prep  Cleaner
     E_1        15       20        1        1           1          0        1
     E_2        15       20        1        1           0          1        1
     E_3        15       20        1        0           1          1        0
     E_4        15       20        1        1           0          0        1
     E_5        15       20        0        1           1          1        0
     E_6        15       20        1        0           0          1        1
     E_7        15       20        1        1           0          0        1
     E_8        15       20        0        1           1          1        0
     E_9        15       20        1        0           1          0        1
    E_10        15       20        1        1           0          1        0
    E_11        15       20        